# Cvičenie 10: Simulácia letu komára

Určite ste už zažili, že ste sa snažili zaspať na konci dlhého letného dňa, no tesne pred tým, že by ste sa ponorili do spánku, ste započuli [iritujúci zvuk komára](https://www.youtube.com/watch?v=t6F160gnzJ4). Zvuk je niekedy intenzívnejši, neskôr ho ledva počujete, ako komár letí v izbe na prvý pohľad úplne náhodne. V skutočnosti sa ale nejedná o náhodný let, komár naviguje pomocou čuchu a riadi sa jednoduchými pravidlami.

Na dnešnom cvičení pomocou objektovo orientovaného programovania a simulácie si ukážeme, ako komár dokáže nájsť svoju korisť vďaka geniality prírody. Na konci cvičenia teda budete mať implementáciu kódu, ktorá nasimuluje let komára nasledujúcim spôsobom (modrá bodka reprezentuje človeka, červený krížik komára):

![Simulácia letu komára](sources/lab10/animation.gif)

Simulácia sa skladá z niekoľkých konceptov, a to konkrétne:

* `Field` - dvojrozmerný priestor, v ktorom sa nachádzajú naši aktéri (komár a človek)
* aktéri `Mosquito` a `Human` - reprezentujú entity, ktoré sa pohybujú v priestore; pre zjednotenie funkcionality použijeme aj nadtriedu `Actor` z minulého cvičenia
* pre reprezentáciu pohybu použijeme pomocné triedy `Location`, ktorá reprezentuje polohu aktéra, a `Direction` pre reprezentáciu pohybu a pre výpočet aktualizovanej polohy - obe triedy boli vytvorené na minulom cvičení.

[Stiahnite si kostru riešenia](sources/lab10/lab10.zip) alebo pracujte v tomto notebooku.

## Krok 1: Umiestnenie aktérov

Z minulého týždňa máme už pripravené definície polohy a pohybu, avšak zatiaľ nevieme, kde sa simulácia odohráva. Práve preto do nášho modelu pridáme triedu `Field` (v súbore `field.py`), ktorá definuje spôsob narábania s našimi aktérmi. Väčšina funkcionality je už hotová, ostávajú nám iba dve metódy `get_actor_distance` a `found`. Ostatné metódy majú nasledovnú funkcionalitu:

* `move_human`: posunie človeka (podľa definície pohybu v triede `Human`);
* `move_mosquito`: posunie komára (podľa definície pohybu v triede `Mosquito`);
* `get_human_coordinates`: vráti pozíciu človeka;
* `get_mosquito_coordinates`: vráti pozíciu komára;
* `get_human`: vráti objekt človeka;
* `get_mosquito`: vráti objekt komára.

**Úloha:** Chýbajúca metóda `get_actor_distance` má vrátiť vzdialenosť medzi človekom a komárom (použije sa pri definícii pohybu komára) a metóda `found` vráti `boolean` vyjadrujúci, či komár našiel človeka alebo nie. Nájdenie koristi definujeme ako moment, keď vzdialenosť medzi človekom a komárom je menšia ako 2. Pri implementácii nezabudnite použiť už implementované metódy, ak tie sú vhodné pre našu funkcionalitu.

In [ ]:
class Field:
    def __init__(self, human, mosquito):
        self.human = human
        self.mosquito = mosquito

        self.human.set_field(self)
        self.mosquito.set_field(self)

    def move_human(self):
        self.human.move()

    def move_mosquito(self):
        self.mosquito.move()

    def get_human_coordinates(self):
        return self.human.get_coords()

    def get_mosquito_coordinates(self):
        return self.mosquito.get_coords()

    def get_human(self):
        return self.human

    def get_mosquito(self):
        return self.mosquito

    def get_actor_distance(self):
        # TODO: return distance between human and mosquito
        return 0

    def found(self):
        # TODO: return if mosquito has found human (distance between them < 2)
        return False


## Krok 2: Postavy v priestore

V priestore simulácie sa nachádzaju dva typy aktérov: komár a človek. Z minulého cvičenia máme už vytvorenú nadtriedu `Actor`, ktorá deklaruje funkcionalitu aktérov, a to nasledovne:

* `set_field`: pridávanie aktéra do priestoru;
* `move`: (náhodný) pohyb aktéra, jednotlivé podtriedy ju bližšie špecifikujú;
* `get_coords`: vráti x-ovú a y-ovú súradnicu aktéra;
* `get_location`: vráti pozíciu aktéra ako objekt typu `Location`.

In [ ]:
class Actor:
    def __init__(self, name, speed, x, y):
        self.name = name
        self.speed = speed
        self.loc = Location(x, y)

        self.field = None

    def set_field(self, field):
        self.field = field

    def move(self):
        angle = random.uniform(0, math.tau)
        direction = Direction(angle)
        dx, dy = direction.move(self.speed)
        self.loc = self.loc.move(dx, dy)

    def get_coords(self):
        return self.loc.get_coords()

    def get_location(self):
        return self.loc

### Krok 2.1: Človek

Najprv implementujeme triedu `Human`, keďže to bude jednoduchá záležitosť. V konštruktore voláme konštruktor nadtriedy s cieľom nastaviť všetky členské premenné tak, aby podporovali prácu s objektom.

Ostáva nám metóda `move`, v ktorej chceme implementovať náhodný pohyb človeka. Podľa príkladu konštruktora použite implementované riešenie z triedy `Actor`, prípadne upravte implementáciu ak to bude potrebné.

In [ ]:
class Human(Actor):
    def __init__(self, name, speed, x, y):
        super().__init__(name, speed, x, y)

    def move(self):
        # TODO: take a step in a random direction
        pass

### Krok 2.2: Komár

Let komára sa riadi niekoľkými jednoduchými pravidlami, ktoré vyžadujú ukladávanie pomocných hodnôt, pre ktoré vytvoríme pomocné členské premenné v konštruktore:

* `found_prey` - reprezentuje, či komár našiel korisť;
* `last_distance` - vzdialenosť od koristi v predošlom kroku;
* `last_direction` - smer posledného kroku, ktorý komár urobil;
* `changes` - zoznam otočení za posledné kroky - budeme ho potrebovať aby sme predišli zacykleniu.

Následne vieme implementovať funkcionalitu metódy `move` na základe nasledovných pravidiel:

* Kým komár je vo väčšej vzdialenosti od človeka ako 90, pohybuje sa náhodne.
* Ak vzdialenosť je menšia, ako 90, komár zachytil vôňu krvi človeka. Od tohto momentu bude jeho pohyb riadený:
  * Najprv začne letieť v náhodnom smere. Ak vôňa je silnejšia (vzdialenosť je menšia), tak pokračuje v tomto smere. V opačnom prípade sa otočí o 180 stupňov.
  * Ak vzdialenosť bude väčšia ako 90, komár stratil vôňu a znova letí náhodným spôsobom.

**Poznámka:** Vzhľadom na to, že v našom riešení komára otočíme vždy o 180 stupňov (čo v skutočnosti nie úplne platí), môže sa stať, že komár bude oscilovať medzi dvomi polohami. Ošetrite túto možnosť a predídte zacykleniu pomocou zoznamu `changes`: ak konár pokračuje v rovnakom smere, pridajte na koniec zoznamu `1`, ak ste komára otočili, tak pridajte `-1`.

In [ ]:
class Mosquito(Actor):
    def __init__(self, name, speed, x, y):
        super().__init__(name, speed, x, y)
        self.found_prey = False
        self.last_distance = 0
        self.last_direction = None
        self.changes = list()

    def move(self):
        # TODO: implement mosquito's movement:
        # while distance > 90 - move in a random direction
        # if distance < 90, the mosquito has found the human scent trace
        # fly in random direction, if scent gets stronger, continue
        # if scent gets weaker, turn around
        # if scent is lost, fly in a random direction
        pass


## Krok 3: Simulácia

Ak ste sa dostali do tohto bodu, už máte všetko pripravené k simulácii. Kód simulácie nájdete v súbore `main.py`, a využíva knižnicu `matplotlib` pre vizualizáciu.

Na začiatku simulácie náhodne vygenerujeme pozíciu človeka, komár začne na pozícii `[10, 10]`. Simulácia trvá až dovtedy, kým komár nenájde človeka, pričom aktualizuje sa iba vykreslený graf, teda pozícia bodov reprezentujúcich človeka (modrý kruh) a komára (červený krížik).

In [ ]:
import random
from time import sleep

import matplotlib.pyplot as plt


human_x = random.randint(-50, 50)
human_y = random.randint(-50, 50)
human = Human("Janko Hrasko", 1, human_x, human_y)
mosquito = Mosquito("Mosquito", 1, 10, 10)

field = Field(human, mosquito)

plt.ion()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim([-100, 100])
ax.set_ylim([-100, 100])

human_pos = field.get_human_coordinates()
mosquito_pos = field.get_mosquito_coordinates()

human_plot_point, = ax.plot(human_pos, 'bo')
mosquito_plot_point, = ax.plot(mosquito_pos, 'rx')
fig.canvas.draw()
fig.canvas.flush_events()

while not field.found():
    human_pos = field.get_human_coordinates()
    mosquito_pos = field.get_mosquito_coordinates()

    human_plot_point.set_xdata(human_pos[0])
    human_plot_point.set_ydata(human_pos[1])
    mosquito_plot_point.set_xdata(mosquito_pos[0])
    mosquito_plot_point.set_ydata(mosquito_pos[1])

    fig.canvas.draw()
    fig.canvas.flush_events()

    field.move_mosquito()
    sleep(0.1)

Ako môžete vidieť, v simulácii sa zatiaľ pohybuje iba komár, pridajte teda pohyb človeka, náhodne vygenerujte aj začiatočnú pozíciu komára, a všeobecne vyskúšajte rôzne nastavenia.

Následne pomocou simulácií odpovedzte na otázku, koľko krokov potrebuje komár na nájdenie koristi v priemere, ak začne v istej vzdialenosti od človeka. Simuláciu implementujte vo funkcii `get_flight_length`, ktorá dostane ako parameter `init_dist` počiatočnú vzdialenosť medzi dvomi aktérmi. V simulácii nepotrebujete vizualizáciu pohybu, nájdenie koristi simulujte iba pomocou výpočtového modelu.

In [ ]:
def get_flight_length(init_dist):
    # TODO: implement simulation and answer the question
    # how many steps it takes for the mosquito to reach its prey
    # on average given an initial distance init_dist
    pass

get_flight_length(100)

## Doplnkové úlohy

1. Cieľom simulácií je nielen namodelovať a nasimulovať jav, ale aj získať užitočné znalosti o skúmanom jave pomocou nich. Navrhnite a následne implementujte simuláciu, ktorá zistí, ako ovplyvňuje počiatočná vzdialenosť komára od koristi počet krokov, ktorý potrebuje na nájdenie koristi.
2. Pridajte do simulácie niekoľko ľudí, pričom pri lete komára berte do úvahy vzdialenosť od najbližšej osoby.
3. Pridajte do sveta prekážky, ktorými komár nedokáže prejsť.